In [22]:
import os
import json
import csv
from multiprocessing import Process
import requests
import pandas as pd

In [23]:
def get_files(f1, f2):
    with open(f1, 'r') as fp:
        json_files = fp.readlines()
    with open(f2, 'r') as fp:
        error_files = fp.readlines()
    json_list = ['index_json/' + f.strip() for f in json_files]
    error_list = [f.strip() for f in error_files]
    return json_list, error_list

In [24]:
json_list, error_list = get_files('json_list.txt', 'Error_files.csv')

In [25]:
def create_json_data(rfile):
    with open(rfile, 'r') as fp:
        try:
            data = json.loads(fp.read())
        except ValueError as ve:
            print('not able to open file: %s'%(rfile))
            data = []
        if len(data) !=  0:
            reporting_data = data['reporting_structure']
            for rpt_dict in reporting_data:
                rpt_plan = rpt_dict['reporting_plans']
                file_data = rpt_dict['in_network_files']
                plan_id = [plan['plan_id'] for plan in rpt_plan]
                yield file_data, plan_id
                

In [26]:
def process_file_data():
    res = {}
    throw = {}
    count = 0
    for file_data, plan_id in create_json_data('index_json/2022-07-24_HUMANA-INC-_index.json'):
        for data in file_data:
            file = data['location'].split('/')[4].split('=')[2]
            if file in res:
                res[file].extend(plan_id)
                pid = set(res[file])
                res[file] = [f for f in pid]
            else:
                res[file] = plan_id
    return res

In [27]:
master_dict = process_file_data()

In [29]:
def create_df_csv(master_dict):
    df_dict = {}
    df_dict['file'] = []
    df_dict['plan_id'] = []
    for file, plan_id in master_dict.items():
        plan_id = (',').join(plan_id)
        df_dict['file'].append(file)
        df_dict['plan_id'].append(plan_id)
    pd.DataFrame(df_dict).to_csv('json_data_3.csv')

In [30]:
def populate_master(json_list):
    for file in json_list:
        for file_data, plan_id in create_json_data(file):
            for data in file_data:
                file = data['location'].split('/')[4].split('=')[2]
                if file in master_dict:
                    master_dict[file].extend(plan_id)
                    pid = set(master_dict[file])
                    master_dict[file] = [f for f in pid]
                else:
                    master_dict[file] = plan_id
    create_df_csv(master_dict)

In [32]:
print(len(master_dict))

422472


In [33]:
create_df_csv(master_dict)